In [ ]:
%reload_ext autoreload
%autoreload 2

import pnllib
import pandas as pd
import numpy as np
from positionslib import get_expiration_indexed_positions
pnllib.set_log_level(pnllib.ERROR)

In [ ]:
NEEDED_COLUMNS=['Symbol', 'Quantity', 'Cost Basis', "PnL", "Margin", "Margin Requirement"]

filename="/Users/rameshsubrahmanyam/Desktop/pos.csv"

dct=get_expiration_indexed_positions(filename, needed_columns=NEEDED_COLUMNS)

# Convert dictionary to DataFrame
rows = []
for date, df in dct.items():
    row = {
        'Date': date,
        'Cost Basis': int(-df['Cost Basis'].sum()),
        'Quantity': int(-df['Quantity'].sum()),
        'Premium': int(-df['Cost Basis'].sum().round()),  # Premium = Cost Basis
        'PnL': int(df['PnL'].sum()),
        'Margin': int(df['Margin'].sum())
    }
    rows.append(row)

# Create DataFrame indexed by date
weekly_df = pd.DataFrame(rows)
weekly_df['Date'] = pd.to_datetime(weekly_df['Date'])
weekly_df = weekly_df.set_index('Date')
weekly_df = weekly_df.sort_index()

print("Weekly DataFrame:")
print(weekly_df)
print("\n")

# Create monthly aggregated DataFrame
monthly_df = weekly_df.copy()
monthly_df['Month'] = monthly_df.index.to_period('M')
monthly_agg = monthly_df.groupby('Month').agg({
    'Cost Basis': 'sum',
    'Quantity': 'sum',
    'Premium': 'sum',
    'PnL': 'sum',
    'Margin': 'sum'
})

print("Monthly Aggregated DataFrame:")
print(monthly_agg)

In [ ]:
d=dct["11/14/2025"]
d["gap"]=-d["Cost Basis"] - d.PnL
d.loc[d.PnL + d["Cost Basis"] < -25, ["Symbol", "Quantity", "Cost Basis", "PnL", "gap", "Margin"] ].reset_index()

In [ ]:
Total margin 117241
Weekly DataFrame:
            Cost Basis  Quantity  Premium   PnL  Margin
Date                                                   
2025-11-14        1580        28     1581  1197   10545
2025-11-21        7420       111     7421  3654   24961
2025-11-28        2229        38     2230  1350   11518
2025-12-05        2881        49     2882   192   15560
2025-12-12        2894        44     2895  -220   13875
2025-12-19        5371        89     5371  -945   29778
2025-12-26         306         5      307   -85    2047


Monthly Aggregated DataFrame:
         Cost Basis  Quantity  Premium   PnL  Margin
Month                                               
2025-11       11229       177    11232  6201   47024
2025-12       11452       187    11455 -1058   61260

3588

